<a href="https://colab.research.google.com/github/Saurav-Raghaw/DataScience/blob/main/Recommeder_System_Surprise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Surprise: Recommender System

#Importing Liabraries.


In [ ]:
!pip install surprise 

     |████████████████████████████████| 11.8MB 227kB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1617618 sha256=70bd4dd167617f33ad9e2c8c931793860f14ff5a75cda6402150fb575a470018
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [ ]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate # it only takes a few lines of code to run a cross-validation procedure.

##About surprise:

* Surprise is a Python scikit for building and analyzing recommender systems that deal with explicit rating data.

* We can use both built-in datasets (Movielens, Jester), and our own custom datasets.

* This module provide various ready-to-use prediction algorithms such as baseline algorithms, neighborhood methods, matrix factorization-based ( SVD, PMF, SVD++, NMF), and many others. Also, various similarity measures (cosine, MSD, pearson…) are built-in.

* The name SurPRISE (roughly :) stands for Simple Python RecommendatIon System Engine.

* Please note that surprise does not support implicit ratings or content-based information.

* https://surprise.readthedocs.io/en/stable/getting_started.html



##1.Loading the Dataset.

In [ ]:
# Load the movielens-100k dataset (download it if needed),
data = Dataset.load_builtin('ml-100k')

#The load_builtin() method will offer to download the movielens-100k dataset if it has not already been downloaded.

Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [ ]:
#SVD algorithm may choose other algorithms also.
algo = SVD()

# algo = KNNBasic()

In [ ]:
# Run 5-fold cross-validation and print results
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

#The cross_validate() function runs a cross-validation procedure according to the cv argument, and computes some accuracy measures.

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9367  0.9366  0.9395  0.9291  0.9347  0.9353  0.0035  
MAE (testset)     0.7378  0.7359  0.7387  0.7338  0.7368  0.7366  0.0017  
Fit time          5.01    4.99    5.06    4.92    4.91    4.98    0.06    
Test time         0.18    0.26    0.24    0.18    0.24    0.22    0.03    


{'fit_time': (5.009145736694336,
  4.990138530731201,
  5.060184955596924,
  4.917062759399414,
  4.907081604003906),
 'test_mae': array([0.73775614, 0.73592771, 0.738676  , 0.7338218 , 0.73676448]),
 'test_rmse': array([0.93674705, 0.93664724, 0.9395076 , 0.92905269, 0.93466287]),
 'test_time': (0.1794724464416504,
  0.2647058963775635,
  0.2437272071838379,
  0.18080997467041016,
  0.23756837844848633)}

##Train-test split and the fit() method.

* If we don’t want to run a full cross-validation procedure, we can use the train_test_split() to sample a trainset and a testset with given sizes, and use the accuracy metric of our choice.

In [ ]:
from surprise import accuracy
from surprise.model_selection import train_test_split

In [ ]:
# sample random trainset and testset
# test set is made of 25% of the ratings.

trainset, testset = train_test_split(data, test_size=.25)

In [ ]:
# Training of the algorithm on the trainset, and predict ratings for the testset.

algo.fit(trainset)
predictions = algo.test(testset)

#Training and Testing together.
#predictions = algo.fit(trainset).test(testset)

In [ ]:
# RMSE
accuracy.rmse(predictions)

RMSE: 0.9408


0.9408175468343966

##Train on a whole trainset and the predict() method.

* We can simply fit our algorithm to the whole dataset, rather than running cross-validation.
* This can be done by using the build_full_trainset() method which will build a trainset object.


In [ ]:
from surprise import KNNBasic


In [ ]:
# Retrieve the trainset.
trainset = data.build_full_trainset()

In [ ]:
# Build an algorithm, and train it.

algo = KNNBasic()
algo.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [ ]:
#We can now predict ratings by directly calling the predict() method. 
#user 196
#item 302
#true rating rui=4.

uid = str(196)  # raw user id (as in the ratings file). They are **strings**!
iid = str(302)  # raw item id (as in the ratings file). They are **strings**!

# get a prediction for specific users and items.
pred = algo.predict(uid, iid, r_ui=4, verbose=True) #The predict() uses raw ids.

user: 196        item: 302        r_ui = 4.00   est = 4.06   {'actual_k': 40, 'was_impossible': False}


#Building the RS Model Using Custom Dataset.

* Loading a rating dataset can be done either from a file (e.g. a csv file), or from a pandas dataframe. 
* Either way, we will need to define a Reader object for Surprise to be able to parse the file or the dataframe.

* To load a dataset from a file (e.g. a csv file), we will need the load_from_file() method.



In [ ]:
import os
from surprise import BaselineOnly
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate

In [ ]:
# As we're loading a custom dataset, we need to define a reader. In the
# movielens-100k dataset, each line has the following format:
# 'user item rating timestamp', separated by '\t' characters.

reader = Reader(line_format='user item rating timestamp', sep='\t')

In [ ]:
data = Dataset.load_from_file('/content/u.data', reader=reader)

In [ ]:
# We can now use this dataset as we please, e.g. calling cross_validate
cross_validate(BaselineOnly(), data, verbose=True)

#https://surprise.readthedocs.io/en/stable/basic_algorithms.html

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating RMSE, MAE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9520  0.9441  0.9345  0.9498  0.9380  0.9437  0.0067  
MAE (testset)     0.7557  0.7494  0.7375  0.7542  0.7442  0.7482  0.0067  
Fit time          0.63    0.44    0.43    0.44    0.46    0.48    0.07    
Test time         0.25    0.11    0.20    0.12    0.11    0.16    0.06    


{'fit_time': (0.6259043216705322,
  0.43619251251220703,
  0.4311187267303467,
  0.4371979236602783,
  0.4615774154663086),
 'test_mae': array([0.75574315, 0.74936406, 0.73753562, 0.75415727, 0.7441604 ]),
 'test_rmse': array([0.95201347, 0.94409266, 0.93447313, 0.94976495, 0.93796566]),
 'test_time': (0.24997425079345703,
  0.10508298873901367,
  0.2003333568572998,
  0.1164541244506836,
  0.1079111099243164)}

# Loading a Dataset from a pandas Dataframe.

In [ ]:
#load_from_df() method.

import pandas as pd

from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate

In [ ]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

df = pd.read_csv('u.data', sep='\t', names=r_cols,encoding='latin-1')
df.head()

,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column          Non-Null Count   Dtype
---  ------          --------------   -----
 0   user_id         100000 non-null  int64
 1   movie_id        100000 non-null  int64
 2   rating          100000 non-null  int64
 3   unix_timestamp  100000 non-null  int64
dtypes: int64(4)
memory usage: 3.1 MB


In [ ]:
# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(1, 5))

In [ ]:
# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df[['user_id', 'movie_id', 'rating']], reader)

In [ ]:
# We can now use this dataset as we please, e.g. calling cross_validate
cross_validate(NormalPredictor(), data, cv=2)

{'fit_time': (0.08648490905761719, 0.11916112899780273),
 'test_mae': array([1.22570948, 1.22118597]),
 'test_rmse': array([1.52599025, 1.51941042]),
 'test_time': (0.5021884441375732, 0.47644853591918945)}

In [ ]:
algo = SVD()
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9299  0.9370  0.9348  0.9340  0.9434  0.9358  0.0044  
MAE (testset)     0.7339  0.7390  0.7378  0.7373  0.7430  0.7382  0.0029  
Fit time          4.86    4.84    4.79    4.80    4.80    4.82    0.03    
Test time         0.16    0.26    0.17    0.25    0.26    0.22    0.05    


{'fit_time': (4.85735559463501,
  4.837671518325806,
  4.788275718688965,
  4.798793077468872,
  4.797749042510986),
 'test_mae': array([0.73389366, 0.73896972, 0.73777651, 0.7372644 , 0.74300308]),
 'test_rmse': array([0.92987027, 0.93697374, 0.93479232, 0.93400538, 0.94342304]),
 'test_time': (0.15524673461914062,
  0.2642784118652344,
  0.1736288070678711,
  0.2541377544403076,
  0.2642521858215332)}

In [ ]:
#We can now predict ratings by directly calling the predict() method. 
#user 196
#item 302
#true rating rui=4.

uid = 166  # raw user id: here uid is integer.
iid = 346  # raw item id: here iid is also integer.

# get a prediction for specific users and items.
pred = algo.predict(uid, iid, r_ui=1, verbose=True) #The predict() uses raw ids.

user: 166        item: 346        r_ui = 1.00   est = 3.73   {'was_impossible': False}


In [ ]:
algo = KNNBasic()
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9740  0.9811  0.9761  0.9812  0.9833  0.9791  0.0035  
MAE (testset)     0.7687  0.7772  0.7709  0.7725  0.7766  0.7732  0.0033  
Fit time          0.61    0.54    0.47    0.45    0.49    0.51    0.06    
Test time         4.63    5.01    3.90    3.93    3.90    4.27    0.46    


{'fit_time': (0.6117043495178223,
  0.5368187427520752,
  0.47462034225463867,
  0.4498016834259033,
  0.4903712272644043),
 'test_mae': array([0.76868022, 0.7771888 , 0.77093734, 0.77249274, 0.77659225]),
 'test_rmse': array([0.97403091, 0.98108918, 0.9760672 , 0.98120657, 0.98330198]),
 'test_time': (4.630460977554321,
  5.012627124786377,
  3.8999826908111572,
  3.928605556488037,
  3.8999552726745605)}

In [ ]:
#We can now predict ratings by directly calling the predict() method. 
#user 196
#item 302
#true rating rui=4.

uid = 196  # raw user id: integer
iid = 302  # raw item id: integer

# get a prediction for specific users and items.
pred = algo.predict(uid, iid, r_ui=4, verbose=True) #The predict() uses raw ids.

user: 196        item: 302        r_ui = 4.00   est = 3.89   {'actual_k': 40, 'was_impossible': False}


##Cross-validation iterators.
* For cross-validation, we can use the cross_validate() function.

* For a better control, we can also instanciate a cross-validation iterator, and make predictions over each split using the split() method of the iterator, and the test() method of the algorithm.

In [ ]:
#classical K-fold cross-validation procedure with 3 splits.

from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import KFold

In [ ]:
# Loading the movielens-100k dataset
data = Dataset.load_builtin('ml-100k')

In [ ]:
# defining a cross-validation iterator
kf = KFold(n_splits=3)

In [ ]:
algo = SVD()

In [ ]:
for trainset, testset in kf.split(data):

  # train and test algorithm.
  algo.fit(trainset)
  predictions = algo.test(testset)

  # Compute and print Root Mean Squared Error
  accuracy.rmse(predictions, verbose=True)

RMSE: 0.9523
RMSE: 0.9419
RMSE: 0.9445
